In [1]:
import geopyspark as gps
import matplotlib.pyplot as plt

from colortools import Color
from pyspark import SparkContext
from functools import partial

import fiona
import pyproj

from shapely.geometry import MultiPoint, MultiLineString, shape
from shapely.ops import transform
from geonotebook.wrappers import VectorData, TMSRasterData
%matplotlib inline

In [2]:
!curl -o /tmp/cropped.tif https://raw.githubusercontent.com/tpolong/geodocker/master/cropped.tif

In [3]:
conf = gps.geopyspark_conf(master="local[*]", appName="visualization")
pysc = SparkContext(conf=conf)

In [4]:
raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="/tmp/cropped.tif")
tiled_layer = raster_layer.tile_to_layout(layout=gps.GlobalLayout(), target_crs=3857)

In [5]:
pyramided = tiled_layer.pyramid()
pyramided

In [6]:
gps.Pyramid([tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)])

In [7]:
gps.Pyramid({x: tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)})

In [8]:
hist = pyramided.get_histogram()
hist

In [9]:
pyramided + 1

In [10]:
(2 * (pyramided + 2)) / 3

In [11]:
small_pyramid = gps.Pyramid({x: tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 5)})
small_pyramid

In [12]:
pyramided + small_pyramid

In [13]:
color_ramp = gps.get_colors_from_matplotlib(ramp_name="viridis")
color_ramp

In [14]:
gps.get_colors_from_matplotlib(ramp_name="hot", num_colors=150)

In [15]:
colors = [Color('green'), Color('red'), Color('blue')]
colors

[Color(0, 128, 0, 255), Color(255, 0, 0, 255), Color(0, 0, 255, 255)]

In [16]:
colors_color_ramp = gps.get_colors_from_colors(colors=colors)
colors_color_ramp

[8388863, 4278190335, 65535]

In [17]:
gps.ColorMap.from_histogram(histogram=hist, color_list=color_ramp)

In [18]:
gps.ColorMap.from_colors(breaks=[0, 250, 1000], color_list=colors_color_ramp)

In [19]:
gps.ColorMap.nlcd_colormap()

In [20]:
break_map = {
    1: 0x00000000,
    2: 0x00000001,
    3: 0x00000002
}

gps.ColorMap.from_break_map(break_map=break_map)

In [21]:
# build using a Histogram
gps.ColorMap.build(breaks=hist, colors=color_ramp)

# It is also possible to pass in the name of Matplotlib color ramp instead of constructing it yourself
gps.ColorMap.build(breaks=hist, colors="viridis")

# build using Colors
gps.ColorMap.build(breaks=colors_color_ramp, colors=colors)

# buld using breaks
gps.ColorMap.build(breaks=break_map)

In [22]:
M.set_center(80.546,6.585, 11)

In [36]:
weighted_histogram = gps.ColorMap.from_histogram(histogram=hist, color_list=color_ramp)
weighted_color_map =gps.ColorMap.nlcd_colormap()

In [37]:
tms = gps.TMS.build(source=pyramided ,
                    display=weighted_color_map)

In [38]:
M.add_layer(TMSRasterData(tms), name="Weighted Layer")